In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import random as rand

In [2]:
'''
STEP 1: LOADING DATASET
'''
 
train_dataset = dsets.MNIST(root='./MNIST-data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
 
test_dataset = dsets.MNIST(root='./MNIST-data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [3]:
class LinearModel(nn.Module):
    
    def __init__(self, input_dim, output_dim, 
                algorithm_name='LogisticRegression',
                train_batch_size=1000, num_epochs=10,
                learning_rate=0.001, optimizer=torch.optim.Adam):
        
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        parameters = self.linear.parameters()
        algorithm_to_loss = {
            'LogisticRegression': nn.CrossEntropyLoss(),
            'SVM': self.hingeLoss}
        
        if not any(algorithm_name==algorithm
                   for algorithm in algorithm_to_loss.keys()):
            raise ValueError ('{} not currently supported', algorithm_name)
        else:
            self.criterion = algorithm_to_loss[algorithm_name]
        
        self.batch_size = train_batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.optimizer = optimizer
             
    def forward(self, x):
        return self.linear(x)
    
    def hingeLoss(self, logits, targets):
        
        zero = torch.Tensor([0]) 
        targets = targets.long()
        score_target_class = logits.gather(1, targets.view(-1,1))  # scores of the target class
        targets = targets.view(-1,1)
        ones = torch.Tensor([1])
        loss = torch.sum(torch.max(zero, logits - score_target_class + 1.0), dim =1)
        loss = loss - ones
        loss = torch.mean(loss) # Getting the mean loss
        return loss
    
    def fit_and_evaluate(self, model, train_dataset, test_dataset):
        
        optimizer = self.optimizer(model.parameters(), lr=self.learning_rate)
        
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=self.batch_size, 
                                           shuffle=True)
        
        for epoch in range(self.num_epochs):
            for images, labels in train_loader:
                images = Variable(images.view(-1, 28*28))
                labels = Variable(labels)

                # Clear gradients w.r.t. parameters
                optimizer.zero_grad()

                # Forward pass to get output/logits
                outputs = self.forward(images)

                # Calculate Loss: softmax --> cross entropy loss
                loss = self.criterion(outputs, labels)

                # Getting gradients w.r.t. parameters
                loss.backward()

                # Updating parameters
                optimizer.step()
            
            test_images = Variable(test_dataset.test_data.view(-1, 28*28))
            outputs = self.forward(test_images.type(torch.FloatTensor))

            # Get predictions from the maximum value
            _, predicted = torch.max(outputs.data, 1)

            accuracy = 100 * (predicted.cpu() == 
                              test_dataset.test_labels.cpu()).sum() / (
                                len(test_dataset.test_labels))

            # Print Loss
            print('Epoch: {}. Training Loss: {}. Test Accuracy: {}'.format(
                            epoch+1, loss.data, accuracy))
           
    def perform_adversarial_attack(self, model):
        pass

In [4]:
input_dim = 28*28
output_dim = 10

In [5]:
model = LinearModel(input_dim, output_dim, algorithm_name='LogisticRegression')
model.fit_and_evaluate(model, train_dataset, test_dataset)

Epoch: 1. Training Loss: 0.9019647240638733. Test Accuracy: 83
Epoch: 2. Training Loss: 0.6284455060958862. Test Accuracy: 86
Epoch: 3. Training Loss: 0.5312653183937073. Test Accuracy: 88
Epoch: 4. Training Loss: 0.4383023679256439. Test Accuracy: 89
Epoch: 5. Training Loss: 0.4116615056991577. Test Accuracy: 89
Epoch: 6. Training Loss: 0.4236544370651245. Test Accuracy: 90
Epoch: 7. Training Loss: 0.35918644070625305. Test Accuracy: 90
Epoch: 8. Training Loss: 0.3799954354763031. Test Accuracy: 90
Epoch: 9. Training Loss: 0.3798162639141083. Test Accuracy: 90
Epoch: 10. Training Loss: 0.34268519282341003. Test Accuracy: 91


In [6]:
model = LinearModel(input_dim, output_dim, algorithm_name='SVM')
model.fit_and_evaluate(model, train_dataset, test_dataset)

Epoch: 1. Training Loss: 1.2214453220367432. Test Accuracy: 85
Epoch: 2. Training Loss: 0.8400567173957825. Test Accuracy: 88
Epoch: 3. Training Loss: 0.6822003722190857. Test Accuracy: 89
Epoch: 4. Training Loss: 0.6131965517997742. Test Accuracy: 90
Epoch: 5. Training Loss: 0.5730751156806946. Test Accuracy: 90
Epoch: 6. Training Loss: 0.5231001973152161. Test Accuracy: 90
Epoch: 7. Training Loss: 0.43371620774269104. Test Accuracy: 90
Epoch: 8. Training Loss: 0.4522427022457123. Test Accuracy: 90
Epoch: 9. Training Loss: 0.44551682472229004. Test Accuracy: 90
Epoch: 10. Training Loss: 0.4999028444290161. Test Accuracy: 90
